<span>
<img src="http://ash.readthedocs.io/en/latest/_static/ash.png" width="260px" align="right"/>
</span>
<span>
<b>Author:</b> <a href="https://andreafailla.github.io">Andrea Failla</a><br/>
<b>Python version:</b>  3.9<br/>
<b>ASH version:</b>  0.1.0<br/>
<b>Last update:</b> July 2025
</span>

# ASH Walks

In [42]:
#!pip install -e ../

<a id="introduction"></a>
## Introduction

The ASH (Attributed Stream Hypergraph) allows to compute walks on both static and temporal hypergraphs.

This tutorial will guide you through walk‑based algorithms provided by the library.


<a id="basic-concepts"></a>
## Basic Concepts

<a id="what-is-a-hypergraph-walk"></a>
### What is a Hypergraph walk?
A **hypergraph walk** generalises the familiar idea of a graph walk to hypergraphs by tracing a sequence of *hyperedges* rather than individual vertices.

For a positive integer $s$, an $s$-walk of length $k$ between hyperedges $f$ and $g$ is a sequence  

$$
f = e_{i_0},\; e_{i_1},\; \dots,\; e_{i_k}=g
$$

such that each consecutive pair overlaps in **at least $s$ vertices**, i.e.  
$|e_{i_{j-1}} \cap e_{i_j}| \ge s$ for every $j = 1,\dots,k$.

Setting $s = 1$ recovers ordinary graph walks; larger $s$ values define walks that have **no counterpart in graphs** and are unique to hypergraphs.


[🔝 To top](#table-of-contents)


### What is a Time-respecting *s*-walk?  

In a temporal hypergraph, s-walks must also respect temporal constraints.

A **time-respecting *s*-walk** of **length** *k*, **width** *s*, and **duration** *d* is an s-walk where the timestamps of subsquent hyperedges are non-decreasing: $t_i \le t_{i+1}$.

When \(s=1\) the notion collapses to the familiar time-respecting path on a graph; larger *s* enforce thicker overlaps and highlight genuinely higher-order, temporally feasible routes.

[🔝 To top](#table-of-contents)

## Static s-walks
We start with a small handcrafted static hypergraph so results are transparent.

We will manually add a few hyperedges so that multiple s-walks exist between pairs.

Parameters you will often vary:
- `s`: minimum overlap size
- Source / target hyperedge IDs

[🔝 To top](#table-of-contents)

In [3]:
from ash_model import ASH
from ash_model.paths import shortest_s_walk, all_shortest_s_walks, all_shortest_s_walk_lengths, s_components, s_diameter

H = ASH()
# Create deterministic hyperedges at t=0
H.add_hyperedge([1,2,3], start=0)
H.add_hyperedge([2,3,4], start=0)
H.add_hyperedge([3,4,5], start=0)
H.add_hyperedge([4,5,6], start=0)
H.add_hyperedge([1,3,5], start=0)  # chord-like

print('Hyperedges:', list(H.hyperedges()))
for hid in H.hyperedges():
    print(hid, '->', H.get_hyperedge_nodes(hid))

Hyperedges: ['e4', 'e1', 'e5', 'e2', 'e3']
e4 -> frozenset({4, 5, 6})
e1 -> frozenset({1, 2, 3})
e5 -> frozenset({1, 3, 5})
e2 -> frozenset({2, 3, 4})
e3 -> frozenset({3, 4, 5})


In [6]:
s = 2
src = 'e1'
trg = 'e4'
all_walks = all_shortest_s_walks(H, s, hyperedge_a=src)
print(f"Shortest s-walk {s} from {src} to {trg}:", all_walks[trg])
print('Explicit shortest:', shortest_s_walk(H, s, src, trg))

lengths = all_shortest_s_walk_lengths(H, s, hyperedge_a=src)
print('Length summary to target:', lengths[src][trg])

comps = list(s_components(H, s))
print('Number of s-components:', len(comps))
print('Largest component size (edges):', len(max(comps, key=len)))
print('s-diameter:', s_diameter(H, s))

Shortest s-walk 2 from e1 to e4: ['e1', 'e5', 'e3', 'e4']
Explicit shortest: ['e1', 'e5', 'e3', 'e4']
Length summary to target: 4
Number of s-components: 1
Largest component size (edges): 5
s-diameter: 3


## Time-respecting s-walks (example)
We now create a small temporal instance where hyperedges activate at different times.
We then enumerate time-respecting s-walks and annotate them.

[🔝 To top](#table-of-contents)

In [20]:
from ash_model.paths import time_respecting_s_walks, all_time_respecting_s_walks, annotate_walks
import networkx as nx
from ash_model import ASH
from ash_model.utils import from_networkx_maximal_cliques_list

snapshots = [nx.barabasi_albert_graph(100, 5) for _ in range(3)]
T = from_networkx_maximal_cliques_list(snapshots)
print(T)


s = 2
tr_walk = time_respecting_s_walks(T, s, 'e1', 'e4')
print('One time-respecting s-walk e1 -> e4:', tr_walk)

Attributed Stream Hypergraph
Nodes: 100
Hyperedges: 972
Snapshots: 3
Node attributes: 

One time-respecting s-walk e1 -> e4: defaultdict(<class 'list'>, {('e1', 'e4'): [[TemporalEdge(fr='e1', to='e4', weight=2, tid=0)]]})


In [23]:
all_tr = all_time_respecting_s_walks(T, s=3)
print('All time-respecting s-walks (s=3):', all_tr)

All time-respecting s-walks (s=3): {('e775', 'e697'): [[TemporalEdge(fr='e775', to='e697', weight=3, tid=2)]], ('e775', 'e724'): [[TemporalEdge(fr='e775', to='e724', weight=3, tid=2)]], ('e354', 'e460'): [[TemporalEdge(fr='e354', to='e460', weight=3, tid=1)]], ('e354', 'e349'): [[TemporalEdge(fr='e354', to='e349', weight=3, tid=1)]], ('e354', 'e345'): [[TemporalEdge(fr='e354', to='e345', weight=3, tid=1)]], ('e354', 'e342'): [[TemporalEdge(fr='e354', to='e342', weight=3, tid=1)]], ('e665', 'e671'): [[TemporalEdge(fr='e665', to='e671', weight=3, tid=2)]], ('e665', 'e666'): [[TemporalEdge(fr='e665', to='e666', weight=3, tid=2)]], ('e665', 'e667'): [[TemporalEdge(fr='e665', to='e667', weight=3, tid=2)]], ('e665', 'e347'): [[TemporalEdge(fr='e665', to='e347', weight=3, tid=2)]], ('e665', 'e664'): [[TemporalEdge(fr='e665', to='e664', weight=3, tid=2)]], ('e665', 'e659'): [[TemporalEdge(fr='e665', to='e659', weight=3, tid=2)]], ('e674', 'e347'): [[TemporalEdge(fr='e674', to='e347', weight=3,

## Random walkers
Random walkers can be simulated on the static s-overlap graph (edge-centric) or under temporal constraints.
We show probability extraction and a few sampled walks.

[🔝 To top](#table-of-contents)

In [27]:
from ash_model.paths import random_walk_probabilities, random_walks, time_respecting_random_walks

s = 1
rw_probs, edge_to_index = random_walk_probabilities(T, s, edge=False)
print('Transition matrix shape:', rw_probs.shape)
print('Edge index mapping (partial):', dict(list(edge_to_index.items())[:3]))

sampled = random_walks(T, edge=False, num_walks=3, walk_length=5)
print('Static sampled walks:', sampled)

Transition matrix shape: (100, 100)
Edge index mapping (partial): {1: 0, 37: 1, 33: 2}
Static sampled walks: [[ 1  0 38 65 90]
 [37 48 52 75 60]
 [33  7  4 89 29]
 ...
 [94 75  8  0  1]
 [84 72  2  0  2]
 [78 69 13  4  6]]


In [ ]:
tr_sampled = time_respecting_random_walks(T, s=s, num_walks=3, walk_length=5)
print('Temporal sampled walks:', tr_sampled)

## (Optional) Temporal s-DAG construction
Set `RUN_ADVANCED = True` to build the temporal s-DAG for the toy temporal instance.
This can be useful for downstream algorithms (centralities, embeddings).

[🔝 To top](#table-of-contents)

In [33]:


import networkx as nx
from ash_model.paths import temporal_s_dag
dags = []
for he in T.hyperedges():
    dag, _, _ = temporal_s_dag(T, s, he, None)
    dags.append(dag)
G_nx = nx.compose_all(dags)
print('Temporal s-DAG nodes/edges:', G_nx.number_of_nodes(), G_nx.number_of_edges())

Temporal s-DAG nodes/edges: 1962 43379
